In [10]:
import numpy as np
import scipy.integrate as integrate
import cmath

# --------------------------------------------------
# Heston-Nandi GARCH(1,1) characteristic function
# --------------------------------------------------
def heston_nandi_cf(u, params, S0, r, T, v0):
    """
    Characteristic function for log-asset price under Heston-Nandi GARCH model.

    u      : complex argument
    params : dict with keys omega, alpha, beta, gamma, lambd
    S0     : initial stock price
    r      : risk-free rate
    T      : maturity (in years)
    v0     : initial variance
    """
    omega = params['omega']
    alpha = params['alpha']
    beta  = params['beta']
    gamma = params['gamma']
    lambd = params['lambd']  # risk premium

    # Define helper quantities
    xi = alpha + beta + gamma**2
    # Coefficients for recursion
    a  = beta + alpha * gamma**2
    b  = -alpha * gamma

    # Functions C and D satisfy backward recursions; for analytic form use:
    # See Heston-Nandi (2000) closed-form C(T,u) and D(T,u)
    # Here we implement direct time-stepping recursion for simplicity
    dt = T
    C = 0.0 + 0.0j
    D = 0.0 + 0.0j

    # Number of time steps for recursion (can increase for accuracy)
    N = 50
    h = dt / N
    for k in range(N, 0, -1):
        # at step k, time t = k*h
        uk = u
        # recursion formulas (approx Euler step on Riccati ODE):
        dD = -0.5 * uk * (uk + 1j) + (beta + alpha * gamma**2) * D + (-alpha * gamma) * D**2
        dC = 1j * uk * (r) + omega * D
        D += dD * h
        C += dC * h

    # Characteristic exponent
    phi = np.exp(C + D * v0 + 1j * u * np.log(S0))
    return phi

# --------------------------------------------------
# Option pricing via Fourier inversion
# --------------------------------------------------
def call_price_fft(params, S0, r, T, v0, K, alpha=1.5, N=4096, B=1000.0):
    """
    Prices a European call using Carr-Madan FFT under Heston-Nandi GARCH.

    params : Heston-Nandi parameters
    S0     : initial stock price
    r      : risk-free rate
    T      : maturity
    v0     : initial variance
    K      : strike or array of strikes
    alpha  : damping factor > 1
    N      : number of FFT points
    B      : integration limit
    """
    # Grid for integration variable v
    dv = B / N
    v = np.arange(N) * dv

    # Simpson's weights
    SimpsonW = np.ones(N)
    SimpsonW[1::2] = 4
    SimpsonW[2::2] = 2
    SimpsonW[0] = 1
    SimpsonW[-1] = 1

    # Compute the Fourier transform of damped call price
    # psi(v) = exp(-r T) * phi(v - (alpha+1)i) / (alpha^2 + alpha - v^2 + i (2 alpha + 1) v)
    phi_vals = np.array([heston_nandi_cf(vi - 1j * (alpha + 1), params, S0, r, T, v0) for vi in v])
    numerator = np.exp(-r * T) * phi_vals
    denom = alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v
    psi = numerator / denom

    # FFT input
    x = np.exp(1j * B * v / 2) * psi * dv * SimpsonW / 3.0
    # Perform FFT
    fft_vals = np.fft.fft(x)

    # Strike grid in log-space
    ksi = np.log(K / S0)
    dk = 2 * np.pi / B
    k_arr = -N / 2 * dk + np.arange(N) * dk

    # Interpolate to requested strike(s)
    call_prices = np.zeros_like(np.atleast_1d(K), dtype=float)
    for idx, Ki in enumerate(np.atleast_1d(K)):
        xi = np.log(Ki / S0)
        # find nearest grid point
        j = np.argmin(np.abs(k_arr - xi))
        call_prices[idx] = np.exp(-alpha * xi) / np.pi * np.real(fft_vals[j])

    return call_prices if call_prices.size > 1 else call_prices[0]

# --------------------------------------------------
# Wrapper for convenience
# --------------------------------------------------
def price_call_heston_nandi(S0, K, T, r, v0, params, **fft_kwargs):
    """
    Computes call price under Heston-Nandi GARCH via FFT.

    S0, K  : initial price, strike
    T      : maturity
    r      : risk-free rate
    v0     : initial variance
    params : dict of Heston-Nandi parameters
    fft_kwargs : optional FFT parameters (alpha, N, B)
    """
    return call_price_fft(params, S0, r, T, v0, K, **fft_kwargs)

# Example usage:
params = {'omega':1e-6, 'alpha':0.1, 'beta':0.8, 'gamma':-0.2, 'lambd':0.0}
price = price_call_heston_nandi(100, 100, 1.0, 0.01, 0.04, params)
print("Call price:", price)


Call price: nan


/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_96319/3886040056.py:51: RuntimeWarning: overflow encountered in exp
  phi = np.exp(C + D * v0 + 1j * u * np.log(S0))
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_96319/3886040056.py:45: RuntimeWarning: overflow encountered in scalar power
  dD = -0.5 * uk * (uk + 1j) + (beta + alpha * gamma**2) * D + (-alpha * gamma) * D**2
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_96319/3886040056.py:45: RuntimeWarning: invalid value encountered in scalar multiply
  dD = -0.5 * uk * (uk + 1j) + (beta + alpha * gamma**2) * D + (-alpha * gamma) * D**2
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_96319/3886040056.py:47: RuntimeWarning: invalid value encountered in scalar multiply
  D += dD * h
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_96319/3886040056.py:85: RuntimeWarning: invalid value encountered in multiply
  numerator = np.exp(-r * T) * phi_vals


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import skew, kurtosis
from python_module.pricing_model import BlackScholesModel, HestonModel

In [3]:
def corrado_su_call(S, K, T, r, sigma, skew=0, kurt=0):
    """
    S     : Spot price
    K     : Strike price
    T     : Time to maturity (in years)
    r     : Risk-free interest rate
    sigma : Volatility
    skew  : Standardized skewness
    kurt  : Excess kurtosis (kurtosis - 3)
    """

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    phi_d1 = norm.pdf(d1)

    # Black-Scholes price
    C_bs = S * N_d1 - K * np.exp(-r * T) * N_d2

    # Corrado-Su skewness and kurtosis corrections
    skew_term = (1 / 6) * skew * sigma * np.sqrt(T) * S * phi_d1 * (2 - d1**2)
    kurt_term = (1 / 24) * kurt * (sigma * np.sqrt(T))**2 * S * phi_d1 * (d1**3 - 3*d1)

    # Total Corrado-Su price
    C_cs = C_bs + skew_term + kurt_term

    return C_cs

In [4]:
S0 = 100                # Initial stock price
K = 100                 # Strike price
T = 252/252                   # Time to maturity in years
r = 0.00                # Risk-free interest rate
v0 = 0.2**2             # Initial volatility
theta = 0.2**2          # Long-term volatility
kappa = 1.00            # Rate of mean reversion
xi = 0.5                # Volatility of volatility
rho = -0.9              # Correlation between stock and volatility
n_simulations = 1000   # Number of Monte Carlo simulations
n_steps = 252           # Number of time steps in the simulation

model = HestonModel()
results = model.compute_monte_carlo(S0, T, r, v0, theta, kappa, xi, rho, n_simulations, n_steps)
df = pd.DataFrame(results[0])
log_returns = np.log(df).diff().dropna().iloc[:, 0].to_numpy().flatten()

In [5]:
results = dict()
for i in df.columns:
    log_returns = np.log(df).diff().dropna().loc[:, i].to_numpy().flatten()
    for K in [80, 100, 120]:
        corrado_market_price = corrado_su_call(100, K, 252, 0, np.std(log_returns), skew(log_returns), kurtosis(log_returns))
        corrado_iv = BlackScholesModel.solve_sigma(S0, K, T, r, corrado_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
        results[(i, K)] = corrado_iv

/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  sigma = sigma - ((price - market_price) / vega)
/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:14: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  sigma = sigma - ((price - market_price) / vega)
/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:14: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:55: RuntimeWarning: divide by 

In [9]:
results

{(0, 80): 0.2328628752249625,
 (0, 100): 0.24286497211468996,
 (0, 120): 0.24350180208873043,
 (1, 80): 0.1244635577959342,
 (1, 100): 0.1661668485343505,
 (1, 120): 0.1517789911148614,
 (2, 80): 0.28348783371012776,
 (2, 100): 0.2938698880968041,
 (2, 120): 0.29577082779784436,
 (3, 80): nan,
 (3, 100): 0.1101190799417359,
 (3, 120): nan,
 (4, 80): nan,
 (4, 100): 0.11858825608257373,
 (4, 120): nan,
 (5, 80): 0.1639930518830123,
 (5, 100): 0.1830081156368103,
 (5, 120): 0.1810157810377931,
 (6, 80): 0.17307878857725045,
 (6, 100): 0.19032636850650111,
 (6, 120): 0.20219339687022833,
 (7, 80): 0.2568556772364941,
 (7, 100): 0.2676994734255013,
 (7, 120): 0.2675433990131673,
 (8, 80): 0.17422486360835424,
 (8, 100): 0.1786874036145223,
 (8, 120): 0.18027519846791276,
 (9, 80): 0.08917913421964382,
 (9, 100): 0.13580408334877517,
 (9, 120): 0.11508499064823827,
 (10, 80): nan,
 (10, 100): 0.1332174008479355,
 (10, 120): 0.09200251980525734,
 (11, 80): 0.31111277884243305,
 (11, 100): 0.

In [6]:
K = 80
heston_market_price = (df.iloc[-1] - K).clip(0).mean()
heston_iv = BlackScholesModel.solve_sigma(S0, K, T, r, heston_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
corrado_market_price = corrado_su_call(100, K, 252, 0, np.std(log_returns), skew(log_returns), kurtosis(log_returns))
corrado_iv = BlackScholesModel.solve_sigma(S0, K, T, r, corrado_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
print(heston_iv, corrado_iv)


/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  sigma = sigma - ((price - market_price) / vega)
/Users/kevinlaventure/Github/Quantitative_Methods_and_Analysis/python_module/pricing_model.py:14: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))


0.23730357697711835 nan


In [7]:
K = 100
heston_market_price = (df.iloc[-1] - K).clip(0).mean()
heston_iv = BlackScholesModel.solve_sigma(S0, K, T, r, heston_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
corrado_market_price = corrado_su_call(100, K, 252, 0, np.std(log_returns), skew(log_returns), kurtosis(log_returns))
corrado_iv = BlackScholesModel.solve_sigma(S0, K, T, r, corrado_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
print(heston_iv, corrado_iv)


0.17018681754911355 0.12465887787165891


In [8]:
K = 110
heston_market_price = (df.iloc[-1] - K).clip(0).mean()
heston_iv = BlackScholesModel.solve_sigma(S0, K, T, r, heston_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
corrado_market_price = corrado_su_call(100, K, 252, 0, np.std(log_returns), skew(log_returns), kurtosis(log_returns))
corrado_iv = BlackScholesModel.solve_sigma(S0, K, T, r, corrado_market_price, 'call', sigma_init=0.2, tol=1e-5, max_iter=1000)
print(heston_iv, corrado_iv)


0.12863088290395916 0.11818454117511785
